In [ ]:
import os
import scipy.io as sio

# 切换到包含文件的目录
os.chdir('/Users/gengxingri/Desktop/Leetcodes/HUAWEI')

from decimal import Decimal
import numpy as np

from scipy.sparse import csc_matrix,  identity
from scipy.sparse.linalg import eigsh, norm, spsolve, cg, minres
import scipy.sparse as sp
import scipy.sparse.linalg as sla

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import time



In [ ]:
# 读取.mat 文件
data = sio.loadmat('BenElechi1.mat')


A = data['Problem']['A'][0][0]
print(A.shape, type(A))

b = data['Problem']['b'][0][0]
print(b.shape, type(b))



In [ ]:
column = b.getcol(0)
b = column.tocsc() 
print(b.shape, type(b))

In [ ]:
# Calculate the condition number
start_time = time.time()

# return eigenvalues that have the min and max abs values, respectively
eigval_max = eigsh(A, k=1, which='LM', return_eigenvectors=False)[0]
eigval_min = eigsh(A, k=1, which='SM', return_eigenvectors=False, tol=5)[0] # the result is vary rough
print(eigval_min, eigval_max)

condition_number = eigval_max / eigval_min

print("Condition number based on eigenvalues:", condition_number)

end_time = time.time()
execution_time = end_time - start_time

print(f"Runing time：{execution_time} s")

In [ ]:
# Create the diagonal preconditioner
n = A.shape[0]
diag_A = A.diagonal()

D = csc_matrix((1 / np.sqrt(diag_A), (np.arange(n), np.arange(n))), shape=(n, n))

# Apply preconditioning
A_new = D.dot(A.dot(D))
b_new = D.dot(b) / norm(D.dot(b))

print(A_new.diagonal(), norm(b_new))


In [ ]:
# Calculate the condition number

eigval_max = eigsh(A_new, k=1, which='LM', return_eigenvectors=False)[0]
eigval_min = eigsh(A_new, k=1, which='SM', return_eigenvectors=False,tol=5)[0]
print(eigval_min, eigval_max)

precond_cond = eigval_max / eigval_min

print("Condition number based on eigenvalues:", precond_cond)


In [ ]:
# Unique solution of Ax = b

x_unique = spsolve(A_new, b_new)
print(x_unique)

In [ ]:
# CG mtheod

def conjugate_gradient_sparse(A, b, tol=1e-8, max_iter=35000):
    n = b.shape[0] 
    x = np.zeros(n).reshape((n,1))
    r = b - A @ x
    #print(r.shape, type(r))
    xk_cg =[]
    
    
    p = r.copy()
    r_norm_sq = np.dot(r.T,r)
    errors = []  # Initial error
    #print(x.shape, r_norm_sq.shape)

    for i in range(max_iter):
        Ap = A @ p
        alpha = r_norm_sq / np.dot(p.T, Ap)
        x = x + alpha * p
        r -= alpha * Ap
        
        r_norm_sq_new = np.dot(r.T, r)
        
        #print(r_norm_sq_new.shape)
        errors.append(np.sqrt(r_norm_sq_new))  # Append new error
        xk_cg.append(x)
        if np.sqrt(r_norm_sq_new) < tol:
            print(f"Converged after {i+1} iterations")
            return x, errors, xk_cg
        
        beta = r_norm_sq_new / r_norm_sq
        p = r + beta * p
        r_norm_sq = r_norm_sq_new
    
    print(f"Did not converge after {max_iter} iterations")
    return x, errors, xk_cg

# Solve using our CG implementation
start_time = time.time()

b_array = b_new.toarray()
x_cg, errors, xk = conjugate_gradient_sparse(A_new, b_array)

cg_time = time.time() - start_time

print(f"CG method took {cg_time:.4f} seconds")

# change the type of errors_cg and xk_cg
errors_cg = [arr[0][0] for arr in errors]
xk_cg = [np.reshape(arr,-1) for arr in xk]
'''

b_array = b_new.toarray()
errors_cg = []

xk_cg = []

def callback(xk):
    rk = b_array - A_new.dot(xk).reshape(-1,1)
    error_norm = np.linalg.norm(rk)
    errors_cg.append(error_norm)
    xk_cg.append(xk)
    return np.linalg.norm(rk)

x_cg, info_cg = cg(A_new, b_array, tol=1e-8, callback=callback)
'''

print(xk_cg[0:5],len(xk_cg))
print('\n',errors_cg[0:5])

In [ ]:
# MINRES method

errors_minres = []

xk_minres = []

def callback2(xk):
    rk = b_array - A_new.dot(xk).reshape(-1,1)
    error_norm = np.linalg.norm(rk)
    errors_minres.append(error_norm)
    xk_minres.append(xk)
    return np.linalg.norm(rk)

x_minres, info_minres = minres(A_new, b_array, tol=1e-12, callback=callback2)
print(x_minres, info_minres)
print(errors_minres[0:5], '\n', len(errors_minres))

In [ ]:
# Plot the log(||r||/||x||) errors

def plot_log(cg_errors, minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(range(1,len(cg_errors)+1), cg_errors, 'b-', label='CG')
    ax.plot(range(1,len(minres_errors)+1), minres_errors, 'r-.', label='MINRES')

    # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^4$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||r||/||x||)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=22')

    # Set y-axis limits
    ax.set_xticks(np.arange(0, 35001, 5000))
    ax.set_xticklabels([f'{i/10000}' for i in np.arange(0, 35001, 5000)])
    
    ax.set_ylim(-10, 0)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()
    
    
    
xk_cg_norms =  np.array([np.linalg.norm(xk) for xk in xk_cg])
xk_minres_norms =  np.array([np.linalg.norm(xk) for xk in xk_minres])

cg_errors_norm1 = np.log10(errors_cg / xk_cg_norms) 
minres_errors_norm1 = np.log10(errors_minres / xk_minres_norms)


# plot the graph
plot_log(cg_errors_norm1, minres_errors_norm1)

In [ ]:
# Plot the log(||xk-x*||_A) errors

def plot_A_norm(cg_errors, minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(range(1,len(cg_errors)+1), cg_errors, 'b-', label='CG')
    ax.plot(range(1,len(minres_errors)+1), minres_errors, 'r-.', label='MINRES')

     # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^4$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||xk-x*||_A)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=22')

    # Set y-axis limits
    ax.set_xticks(np.arange(0, 35001, 5000))
    ax.set_xticklabels([f'{i/10000}' for i in np.arange(0, 35001, 5000)])
    
    ax.set_ylim(-7, 1)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()

# errors of log||xk - x* ||_A in each iteration
errors_iter_cg = [np.sqrt((xk - x_unique).T.dot(A_new.dot(xk - x_unique))) for xk in xk_cg]
xk_A_norms_cg = [np.array(err) for err in errors_iter_cg]
cg_errors_A_norm = np.log10(xk_A_norms_cg)


# errors of minres
errors_iter_minres = [np.sqrt((xk - x_unique).T.dot(A_new.dot(xk - x_unique))) for xk in xk_minres] # list of array
xk_A_norms_minres = [np.array(err) for err in errors_iter_minres]
minres_errors_A_norm = np.log10(xk_A_norms_minres)


# plot
plot_A_norm(cg_errors_A_norm, minres_errors_A_norm)

In [ ]:
# Plot the log(||xk-x*||) errors

def plot_norm(cg_errors, minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(range(1, len(cg_errors)+1), cg_errors, 'b-', label='CG')
    ax.plot(range(1, len(minres_errors)+1), minres_errors, 'r-.', label='MINRES')

      # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^4$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||xk-x*||)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=22')

    # Set y-axis limits
    ax.set_xticks(np.arange(0, 35001, 5000))
    ax.set_xticklabels([f'{i/10000}' for i in np.arange(0, 35001, 5000)])
    
    ax.set_ylim(-5, 2)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()

# errors of log||xk - x* || in each iteration

# errors of cg
errors3_cg = [np.linalg.norm(xk - x_unique) for xk in xk_cg]
errors_norm3_cg = [np.array(err) for err in errors3_cg]


cg_errors_norm3 = np.log10(errors_norm3_cg)



# errors of minres

errors3_minres = [np.linalg.norm(xk - x_unique) for xk in xk_minres]
errors_norm3_minres = [np.array(err) for err in errors3_minres]


minres_errors_norm3 = np.log10(errors_norm3_minres)



# plot
plot_norm(cg_errors_norm3, minres_errors_norm3)

In [ ]:
# Plot the log(||r||) errors

def plot_norm_4(cg_errors, minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(range(1, len(cg_errors)+1), cg_errors, 'b-', label='CG')
    ax.plot(range(1, len(minres_errors)+1), minres_errors, 'r-.', label='MINRES')

     # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^4$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||r||)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=22')

    # Set y-axis limits
    ax.set_xticks(np.arange(0, 35001, 5000))
    ax.set_xticklabels([f'{i/10000}' for i in np.arange(0, 35001, 5000)])
    
    ax.set_ylim(-9, 0)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()

    
# errors of log||r|| in each iteration

# errors of cg
cg_errors_norm4 = np.log10(errors_cg)



# errors of minres
minres_errors_norm4 = np.log10(errors_minres)



# plot
plot_norm_4(cg_errors_norm4, minres_errors_norm4)

In [ ]:
# Plot the ||x||

def plot_norm_5(cg_errors, minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(range(1,len(cg_errors)+1), cg_errors, 'b-', label='CG')
    ax.plot(range(1,len(minres_errors)+1), minres_errors, 'r-.', label='MINRES')

    # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^4$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('||x||')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=22')

    # Set y-axis limits
    ax.set_xticks(np.arange(0, 35001, 5000))
    ax.set_xticklabels([f'{i/10000}' for i in np.arange(0, 35001, 5000)])
    
    ax.set_ylim(0, 80)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(10))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()
    
    
    
xk_cg_norm5 =  np.array([np.linalg.norm(xk) for xk in xk_cg])
xk_minres_norm5 =  np.array([np.linalg.norm(xk) for xk in xk_minres])

# plot
plot_norm_5(xk_cg_norm5, xk_minres_norm5)

In [ ]:
# Now plot the indefinitesystem (A-\delta I)x = b, where\delta = 0.5

I = identity(n, format='csc')
A_indef = A_new - 0.5*I

# MINRES method

errors_minres_indef = []

xk_minres_indef = []

def callback(xk):
    rk = b_array - A_indef.dot(xk).reshape(-1,1)
    error_norm = np.linalg.norm(rk)
    errors_minres_indef.append(error_norm)
    xk_minres_indef.append(xk)
    return np.linalg.norm(rk)

x_indef_minres, info_indef_minres = minres(A_indef, b_array, tol=1e-11, callback=callback)
print(x_indef_minres, info_indef_minres)

In [ ]:
# Plot the log(||r||) errors

def plot_indef_log(minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    
    ax.plot(range(1,len(minres_errors)+1), minres_errors, 'b-.', label='MINRES')

    # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^5$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||r||)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=48')

     # Set y-axis limits
    ax.set_xticks(np.arange(0, 250001, 50000))
    ax.set_xticklabels([f'{i/100000}' for i in np.arange(0, 250001, 50000)])
    
    ax.set_ylim(-3, 0)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()
    
    
    
errors_indef_minres =  np.array([np.linalg.norm(err) for err in errors_minres_indef])

minres_errors_indef = np.log10(errors_indef_minres)

# plot
plot_indef_log(minres_errors_indef)

In [ ]:
# Plot the ||x|| for the indefinite system

def plot_indef_norm(minres_errors):
    # Create the plot
    fig, ax = plt.subplots(figsize=(10,8))
    
    ax.plot(range(1,len(minres_errors)+1), minres_errors, 'b-.', label='MINRES')

    # Set labels and title
    ax.set_xlabel('iteration count')
    ax.text(1, 0, 'x $10^5$', transform=ax.transAxes, ha='right', va='center')
    
    ax.set_ylabel('log(||r||)')
    ax.set_title('Name:BenElechi_BenElechi1, Dim:245874x245874, nnz:13150496, id=48')

     # Set y-axis limits
    ax.set_xticks(np.arange(0, 250001, 50000))
    ax.set_xticklabels([f'{i/100000}' for i in np.arange(0, 250001, 50000)])
    
    ax.set_ylim(0, 400)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(50))

    # Add legend
    ax.legend()


    # Show the plot
    plt.show()
    
    

xk_minres_indef_norm =  np.array([np.linalg.norm(xk) for xk in xk_minres_indef])

# plot
plot_indef_norm(xk_minres_indef_norm)